In [1]:
import pandas as pd
import meteostat as ms
import h3
from meteostat import Hourly, Point
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import ydata_profiling
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import folium
import time
import plotly.express as px
from sklearn.metrics import mean_squared_error

In [2]:
# Data was distilled to the entry point 200 NM away from the runway of arrival.
# Each aircraft was confirmed to land in LAX no fly overs.
# Each point has specific information (lat, long, ground speed, aircraft type) were used for analysis.
data = pd.read_csv('LAX2m.csv')
data['logTime'] = pd.to_datetime(data['logTime']).dt.tz_localize('UTC')
data['h3_id'] = [h3.geo_to_h3(lat, lon, 2) for lat, lon in zip(data.startlat, data.startlon)]


In [3]:
#Create Figure that shows all entry starting coordinates of data points
fig = px.scatter_mapbox(data, 
                         lat='startlat', 
                         lon='startlon', 
                         title='Starting Lat and Long',
                         hover_name=data.index,  
                         color_discrete_sequence=['blue'],  
                         opacity=0.6,  
                         zoom=1)  

# Calculate the bounds for zooming
lat_min, lat_max = data['startlat'].min(), data['startlat'].max()
lon_min, lon_max = data['startlon'].min(), data['startlon'].max()
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

# Update the layout to zoom in where data exists
fig.update_geos(
    scope='world',
    projection_type='natural earth',
    lataxis_range=[lat_min, lat_max],
    lonaxis_range=[lon_min, lon_max]
)

fig.update_layout(
    mapbox=dict(
        style='open-street-map', 
        center=dict(lat=center_lat, lon=center_lon),
        zoom=4,  
    ),
    title=dict(
        font=dict(size=24, color='darkblue'),
        x=0.5,  
    ),
    margin=dict(l=0, r=0, t=50, b=0)  
)
fig.show()

# Can note where location data concentrates instead of using basic northeast northwest etc.
# I use H3 tiles to distinguish direction of approach.

In [4]:
# Function to get the polygon coordinates for each H3 tile
def get_h3_geojson(h3_id):
    boundary = h3.h3_to_geo_boundary(h3_id, geo_json=True)
    return {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [boundary]
        },
        'properties': {'h3_id': h3_id}
    }
    
h3_ids = list(set(data['h3_id'].to_list()))

# Create a Folium map centered around LAX
lax_location = [33.9416, -118.4085]  # LAX coordinates
m = folium.Map(location=lax_location, zoom_start=12)

# Add H3 tiles to the map
for h3_id in h3_ids:
    geojson_data = get_h3_geojson(h3_id)
    folium.GeoJson(
        geojson_data,
        style_function=lambda x: {
            'fillColor': '#ff7800',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }
    ).add_to(m)

# Display the map
m.save('h3_tiles_lax.html')

In [5]:
# Additional data from Meteostat was used to supplement weather data
# as weather data can have an affect on aircraft travel time.
# Defined the time range for fetching weather data
start_time = '2024-07-01 00:00:00'
start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

end_time = '2024-09-01 00:00:00'
end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')

# LAX Location code for the weather station
location_code = 72295

# hourly weather data for the specified location and date range
weather_data = Hourly(location_code, start=start_time, end=end_time).fetch()

# Reset index to make datetime a column
weather_data.reset_index(inplace=True)

# Convert the 'time' column (which is the datetime) to the same timezone
weather_data['time'] = weather_data['time'].dt.tz_localize('UTC')

# Merge the weather data with the other DataFrame on datetime
merged_data = pd.merge_asof(data.sort_values('logTime'), 
                             weather_data.sort_values('time'), 
                             left_on='logTime', 
                             right_on='time', 
                             direction='backward')


In [6]:
# Remove columns from merged_data that are all nan or missing
merged_data = merged_data.dropna(axis=1, how='all')
# Drop columns that are not needed or not useful for prediciton (if all column is 0 etc.)
drop_cols = ['logTime','startlat','startlon','time','prcp']
merged_data = merged_data.drop(drop_cols, axis=1)


In [7]:
#use ydata_profiling on merged_data
#ydata_profiling gives a streamlined report on feature analysis
profile = ydata_profiling.ProfileReport(merged_data, title="Data Profiling Report", explorative=True)
# Save the report to an HTML file
profile.to_file("data_profiling_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 30.17it/s]


In [8]:
#From the profiling
#AC-role and type have high variation but are correlated to emittercat we can use emittercat in its place
#remove calculated time (this is our baseline) and ttland which is our target
# remove any missing values dwpt, rhum are the only columns with missing values we can drop those rows
merged_data = merged_data.dropna()
drop_cols_2 = ['ACType','AC_role','ttland','calculated_time']
baseline = merged_data[['calculated_time']]
X = merged_data.drop(drop_cols_2, axis=1)
y = merged_data['ttland']

In [9]:
#calculating the MSE between the actual time to land and the calculated time using groundspeed and distance to arrival.
hard_calcMSE = mean_squared_error(y,baseline)
hard_calcr2 = r2_score(y,baseline)
print('Hard Calc MSE:', hard_calcMSE)
print('Hard Calc R2:', hard_calcr2)

Hard Calc MSE: 229.29945981265553
Hard Calc R2: -8.745297428216805


In [10]:
# Get appropriate column headers for categorical and non-categorical columns
ohe = ['emitterCat','h3_id'] #onehot encode categorical columns 
numeric_columns = X.select_dtypes(include='number').columns.tolist() #numeric columns
numeric_columns.remove('emitterCat') #emitterCat is an id therefore categorical


In [11]:
# Split data into test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Preprocess numeric non categorical numbers using scaler
# Preprocess categorical columns using onehotencoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(), ohe)
    ],
    remainder='passthrough' 
)

In [13]:
# A pipeline with preprocessor and Linear Regression for baseline model
basepipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
basepipe.fit(X_train, y_train)
# Make predictions on the test data
predictions = basepipe.predict(X_test)
# Evaluate the model
baseline_linreg_MSE = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", baseline_linreg_MSE)


Mean Squared Error: 4.740309637874746


In [14]:
# Preprocess the data inorder to avoid redundancy of calling preprocessor over and over
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [15]:
# Define models without hyperparameter tuning 
models_base = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),  # Using a default alpha
    'Support Vector Regression': SVR(C=1.0, kernel='rbf', epsilon=0.1),  # Using default params
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, subsample=0.8),
}


In [16]:
## Base Results No Hyperparameter Tuning
results_init = []

for model_name, model in models_base.items():
    start_time = time.time()  # Start timer
    
    # Fit the model
    model.fit(X_train_processed, y_train)
    
    # Predict on the training set and test set
    y_train_pred = model.predict(X_train_processed)
    y_test_pred = model.predict(X_test_processed)
    
    # Calculate MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)

    # Calculate runtime
    runtime = time.time() - start_time
    
    # Append the results to the DataFrame
    results_init.append({
        'Model': model_name,
        'Training MSE': train_mse,
        'Test MSE': test_mse,
        'R2' : r2,
        'Runtime (s)': runtime
    })

# Convert results to a DataFrame
results_init = pd.DataFrame(results_init)

In [23]:
pd.set_option('display.max_colwidth', None)
display(results_init)

,Model,Training MSE,Test MSE,R2,Runtime (s)
0,Linear Regression,5.247364,4.740310,0.792723,0.187235
1,Ridge Regression,5.247551,4.739043,0.792779,0.049623
2,Support Vector Regression,4.811447,4.397559,0.807711,101.546587
3,Gradient Boosting,4.249344,4.088642,0.821218,6.199033
4,XGBoost,4.335094,4.108316,0.820358,0.288604


In [17]:
## Final Model and Hyperparameter Tuning
# Define models and their hyperparameters for GridSearchCV
models = {
    'Ridge Regression': (Ridge(), {
        'alpha': [0.1, 1.0, 10.0],
        'max_iter': [100, 200, 300]
    }),
    'Gradient Boosting': (GradientBoostingRegressor(), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5]
    }),
    'XGBoost': (XGBRegressor(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3,5],
        'subsample': [0.4, 0.8, 1.0]
    }),
}

In [18]:
# Store results in a DataFrame
results_final = []
results_full = {}
for model_name, (model, params) in models.items():
    # Setup GridSearchCV
    grid_search = GridSearchCV(model, params, scoring='neg_mean_squared_error', cv=5)
    start_time = time.time()  # Start timer
    # Fit the model
    grid_search.fit(X_train_processed, y_train)
    # Calculate runtime
    runtime = time.time() - start_time
    # Get the best model and its parameters
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Predict on the training set and test set
    y_train_pred = best_model.predict(X_train_processed)
    y_test_pred = best_model.predict(X_test_processed)
    
    # Calculate MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    best_mse = -grid_search.best_score_
    results_full[model_name] = best_mse
    # Append the results to the DataFrame
    results_final.append({
        'Model': model_name,
        'Best Parameters': best_params,
        'Training MSE': train_mse,
        'Test MSE': test_mse,
        'R2' : r2,
        'Runtime (s)': runtime
    })

# Convert results to a DataFrame
results_final = pd.DataFrame(results_final)

In [24]:
pd.set_option('display.max_rows', None)
display(results_final)

,Model,Best Parameters,Training MSE,Test MSE,R2,Runtime (s)
0,Ridge Regression,"{'alpha': 1.0, 'max_iter': 100}",5.247551,4.739043,0.792779,0.623461
1,Gradient Boosting,"{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}",4.019953,4.011836,0.824577,555.955555
2,XGBoost,"{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}",2.758963,3.852231,0.831556,104.953677


In [20]:
#Compare each model and their baseline performance (no Hyperparameter Tuning)
df_merged = pd.merge(results_init, results_final, on='Model', suffixes=('_Baseline', '_Tuned'))


In [21]:
# Create and save a figure that shows each models baseline and tuned performance along with the linear regression baseline we set earlier
num_models = df_merged.shape[0]
bar_width = 0.35
index = np.arange(num_models)


# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

plt.axhline(y=baseline_linreg_MSE, color='red', linestyle='--', label='Baseline Linear Reg MSE')
# Create bars for baseline and tuned models with specified colors
baseline_bars = ax.bar(index, df_merged['Training MSE_Baseline'], bar_width, label='Baseline', color='darkblue')
tuned_bars = ax.bar(index + bar_width, df_merged['Training MSE_Tuned'], bar_width, label='Tuned', color='lightblue')

# Add labels, title, and custom x-axis tick labels
ax.set_xlabel('Model Name', fontsize=14)
ax.set_ylabel('Training Mean Squared Error (MSE)', fontsize=14)
ax.set_title('Comparison of Training MSE: Baseline vs Tuned Model', fontsize=16)
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(df_merged.Model)
ax.legend()

# Add MSE values on top of bars with adjusted position
for bars in [baseline_bars, tuned_bars]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.01, round(yval, 2), ha='center', va='bottom')

# Show grid
ax.yaxis.grid(True)
plt.tight_layout()
# Display the plot
plt.savefig('mse_comparison_grouped_final.png')  # Save as PNG
plt.show()

C:\Users\JosephCanlas\AppData\Local\Temp\ipykernel_9580\2751276833.py:34: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [22]:
#Final table with all MSE results (hard_calc, baseline, and the final tuned models)
final = pd.DataFrame({"Hard Calc MSE": [hard_calcMSE], "Linear Regression Baseline MSE": [baseline_linreg_MSE],
 "Final Tuned XGBoost MSE": [df_merged['Test MSE_Tuned'].min()]})
display(final)

,Hard Calc MSE,Linear Regression Baseline MSE,Final Tuned XGBoost MSE
0,229.29946,4.74031,3.852231
